# Convert SG Models to CoreML

In [1]:
!python3 -m pip install super-gradients coremltools onnx==1.13.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.2/964.2 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 38.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 k

In [1]:
import os; os.environ["SG_SKIP_ENV_CHECK"] = "1"
from typing import List

import coremltools as ct
from super_gradients.common.object_names import Models
from super_gradients.training import models
import torch

[2023-05-11 14:32:20] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-05-11 14:32:27] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-05-11 14:32:27] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-05-11 14:32:27] WARNING - export.py - Failed to import pytorch_quantization
[2023-05-11 14:32:27] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


## Load any model

In [2]:
print('Loading model...')
sg_torch_model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")

# Defining an example input
example_inputs = [torch.rand(1,3,224,224).cpu()]

# Defining an output path for the compiled CoreML model
output_path = "deci-yolo-nas-s.mlmodel"
print('Loaded successfully')

Loading model...


[2023-05-11 14:32:49] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


  0%|          | 0.00/73.1M [00:00<?, ?B/s]

Downloading: "https://sghub.deci.ai/models/yolo_nas_s_coco.pth" to /root/.cache/torch/hub/checkpoints/yolo_nas_s_coco.pth


In [3]:
def convert_sg_model_to_coreml(sg_model: torch.nn.Module,
                               example_inputs: List[torch.nn.Module],
                               output_path: os.PathLike = None,
                               export_as_ml_program=False):
    """
    Converts a given SG model to CoreML mlprogram or package.

    @param sg_model: A super-gradients nn.Module to compile
    @param example_inputs: A list of input tensors to feed the model, required to trace with torch.jit.
    @param output_path: The path to save the compiled CoreML model into.
    @param export_as_ml_program: Whether to convert to the new program format (better) or legacy coreml proto file (Supports more iOS versions and devices, but this format will be deprecated at some point).
    """
    print('Building model...')
    print(sg_model)
    print('Model child nodes:')
    print(next(sg_model.named_children()))

    # Set the model in evaluation mode.
    sg_model.eval()
    print('Creating torch jit trace...')
    traced_model = torch.jit.trace(sg_model, example_inputs)
    print('Tracing the model with the provided inputs...')
    out = traced_model(*example_inputs)
    print('Inferred output shapes:', [o.shape for o in out])

    print('Converting to CoreML...')
    if not output_path:
        output_path = os.sep.join([os.getcwd(), sg_model.__class__.__name__.lower()])
        output_path += 'mlpackage' if export_as_ml_program else '.mlmodel'

    if export_as_ml_program:
        coreml_model = ct.convert(
            traced_model,
            convert_to="mlprogram",
            inputs=[ct.ImageType(name=f"x_{i+1}",
                                 shape=_.shape) for i, _ in enumerate(example_inputs)]
        )
    else:
        coreml_model = ct.convert(
            traced_model,
            inputs=[ct.ImageType(name=f"x_{i+1}",
                                 shape=_.shape) for i, _ in enumerate(example_inputs)]
        )

    spec = coreml_model.get_spec()
    print(spec.description)

    # Changing the input names:
    #   In CoreML, the input name is compiled into classes (named keyword argument in predict).
    #   We want to re-use the same names among different models to make research easier.
    #   We normalize the inputs names to be x_1, x_2, etc.
    for i, _input in enumerate(spec.description.input):
        new_input_name = "x_" + str(i + 1)
        print(f'Renaming input {_input.name} to {new_input_name}')
        ct.utils.rename_feature(spec, _input.name, new_input_name)

    # Re-Initializing the model with the new spec
    coreml_model = ct.models.MLModel(spec, weights_dir=coreml_model.weights_dir)

    # Saving the model
    coreml_model.save(output_path)
    print('CoreML model successfully save to ', os.path.abspath(output_path))


In [4]:
convert_sg_model_to_coreml(sg_model=sg_torch_model,
                           output_path=output_path,
                           example_inputs=example_inputs,
                           export_as_ml_program=False)

Building model...
YoloNAS_S(
  (backbone): NStageBackbone(
    (stem): YoloNASStem(
      (conv): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        )
        (branch_1x1): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2))
        (post_bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (rbr_reparam): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
    )
    (stage1): YoloNASStage(
      (downsample): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, moment

[2023-05-11 14:33:05] WARNING - torchir_passes.py - Tuple detected at graph output. This will be flattened in the converted model.
[2023-05-11 14:33:05] INFO - converter.py - Converting graph.
[2023-05-11 14:33:05] INFO - builder.py - Adding op 'backbone.stem.conv.branch_3x3.conv.weight' of type const
[2023-05-11 14:33:05] INFO - builder.py - Adding op 'backbone.stem.conv.branch_3x3.bn.running_var' of type const
[2023-05-11 14:33:05] INFO - builder.py - Adding op 'backbone.stem.conv.branch_3x3.bn.running_mean' of type const
[2023-05-11 14:33:05] INFO - builder.py - Adding op 'backbone.stem.conv.branch_3x3.bn.bias' of type const
[2023-05-11 14:33:05] INFO - builder.py - Adding op 'backbone.stem.conv.branch_3x3.bn.weight' of type const
[2023-05-11 14:33:05] INFO - builder.py - Adding op 'backbone.stem.conv.branch_1x1.bias' of type const
[2023-05-11 14:33:05] INFO - builder.py - Adding op 'backbone.stem.conv.branch_1x1.weight' of type const
[2023-05-11 14:33:05] INFO - builder.py - Adding

input {
  name: "x_1"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
    }
  }
}
output {
  name: "var_2557"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
output {
  name: "var_2548"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==1.13.1+cu117"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "6.3.0"
  }
}

Renaming input x_1 to x_1
CoreML model successfully save to  /content/deci-yolo-nas-s.mlmodel


## Visualize with Netron

In [5]:
!python3 -m pip install netron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.5 MB/s eta 0:00:00


In [6]:
import IPython
import threading
import time
import os

def display_netron(path):
    os.system(f'netron {path}')
    
thread = threading.Thread(target=display_netron, args=(output_path,))
thread.start()

time.sleep(1)
display(IPython.display.IFrame(f"http://localhost:8080", width=1000, height=1000))